## Machine Learning 프로젝트 수행을 위한 코드 구조화

- ML project를 위해서 사용하는 템플릿 코드를 만듭니다.

1. **필요한 라이브러리와 데이터를 불러옵니다.**


2. **EDA를 수행합니다.** 이 때 EDA의 목적은 풀어야하는 문제를 위해서 수행됩니다.


3. **전처리를 수행합니다.** 이 때 중요한건 **feature engineering**을 어떻게 하느냐 입니다.


4. **데이터 분할을 합니다.** 이 때 train data와 test data 간의 분포 차이가 없는지 확인합니다.


5. **학습을 진행합니다.** 어떤 모델을 사용하여 학습할지 정합니다. 성능이 잘 나오는 GBM을 추천합니다.


6. **hyper-parameter tuning을 수행합니다.** 원하는 목표 성능이 나올 때 까지 진행합니다. 검증 단계를 통해 지속적으로 **overfitting이 되지 않게 주의**하세요.


7. **최종 테스트를 진행합니다.** 데이터 분석 대회 포맷에 맞는 submission 파일을 만들어서 성능을 확인해보세요.

## 1. 라이브러리, 데이터 불러오기

In [53]:
# 데이터분석 4종 세트
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 모델들, 성능 평가
# (저는 일반적으로 정형데이터로 머신러닝 분석할 때는 이 2개 모델은 그냥 돌려봅니다. 특히 RF가 테스트하기 좋습니다.)
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from xgboost import XGBClassifier

# KFold(CV), partial : optuna를 사용하기 위함
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from functools import partial

# hyper-parameter tuning을 위한 라이브러리, optuna
import optuna

In [54]:
# 데이터를 불러옵니다.
base_path = '../../data/'
# train = pd.read_csv(base_path + 'train.csv', index_col='id')
# test = pd.read_csv(base_path + 'test.csv', index_col='id')
train = pd.read_csv(base_path + 'train_f5.csv', index_col='id')
test = pd.read_csv(base_path + 'test_f5.csv', index_col='id')
submission = pd.read_csv(base_path + 'sample_submission.csv', index_col='id')
print(train.shape, test.shape, submission.shape)

(101763, 23) (67842, 22) (67842, 1)


In [55]:
train['lOBlank_n'] = train['lOBlank_n'].astype('category')
test['lOBlank_n'] = test['lOBlank_n'].astype('category')
train['l_n'] = train['l_n'].astype('category')
test['l_n'] = test['l_n'].astype('category')
train['n_n'] = train['n_n'].astype('category')
test['n_n'] = test['n_n'].astype('category')
train['branchCount_n'] = train['branchCount_n'].astype('category')
test['branchCount_n'] = test['branchCount_n'].astype('category')
train['outlier_count'] = train['outlier_count'].astype('category')
test['outlier_count'] = test['outlier_count'].astype('category')
train['total_Op_n'] = train['total_Op_n'].astype('category')
test['total_Op_n'] = test['total_Op_n'].astype('category')
train['total_Opnd_n'] = train['total_Opnd_n'].astype('category')
test['total_Opnd_n'] = test['total_Opnd_n'].astype('category')

## 2. EDA

- 데이터에서 찾아야 하는 기초적인 내용들을 확인합니다.


- class imbalance, target distribution, outlier, correlation을 확인합니다.

In [56]:
train.columns

Index(['loc', 'v(g)', 'ev(g)', 'iv(g)', 'v', 'd', 'i', 'e', 'b', 't', 'lOCode',
       'lOComment', 'locCodeAndComment', 'uniq_Op', 'uniq_Opnd', 'lOBlank_n',
       'n_n', 'branchCount_n', 'l_n', 'total_Op_n', 'total_Opnd_n',
       'outlier_count', 'defects'],
      dtype='object')

In [57]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 101763 entries, 0 to 101762
Data columns (total 23 columns):
 #   Column             Non-Null Count   Dtype   
---  ------             --------------   -----   
 0   loc                101763 non-null  float64 
 1   v(g)               101763 non-null  float64 
 2   ev(g)              101763 non-null  float64 
 3   iv(g)              101763 non-null  float64 
 4   v                  101763 non-null  float64 
 5   d                  101763 non-null  float64 
 6   i                  101763 non-null  float64 
 7   e                  101763 non-null  float64 
 8   b                  101763 non-null  float64 
 9   t                  101763 non-null  float64 
 10  lOCode             101763 non-null  int64   
 11  lOComment          101763 non-null  int64   
 12  locCodeAndComment  101763 non-null  int64   
 13  uniq_Op            101763 non-null  float64 
 14  uniq_Opnd          101763 non-null  float64 
 15  lOBlank_n          101763 non-null  cat

### 3. 전처리

#### 결측치 처리

### 4. 학습 데이터 분할

In [58]:
# 첫번째 테스트용으로 사용하고, 실제 학습시에는 K-Fold CV를 사용합니다.
from sklearn.model_selection import train_test_split

X = train.drop(columns=['defects'])
y = train.defects

# for OOF-prediction split 5% of data as validation dataset.
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=61, stratify=y)

In [59]:
print(X_train.shape, y_train.shape, X_val.shape, y_val.shape)
print(y_train.mean(), y_val.mean())

(81410, 22) (81410,) (20353, 22) (20353,)
0.2266429185603734 0.22664963396059548


### 5. 학습 및 평가

In [60]:
# 간단하게 LightGBM 테스트
# 적당한 hyper-parameter 조합을 두었습니다. (항상 best는 아닙니다. 예시입니다.)
model = XGBClassifier(
    booster='gbtree',
    objective='binary:logistic',
    max_depth=6,
    learning_rate=0.1,
    n_estimators=100,
    n_jobs=-1,
    random_state=61,
    tree_method='hist',
    enable_categorical=True,
)

In [61]:
print("\nFitting LightGBM...")
model.fit(X_train, y_train)


Fitting LightGBM...


XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=True, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=-1, num_parallel_tree=None,
              predictor=None, random_state=61, ...)

In [62]:
# metric은 그때마다 맞게 바꿔줘야 합니다.
evaluation_metric = roc_auc_score

In [63]:
print("Prediction")
pred_train = model.predict(X_train)
pred_val = model.predict(X_val)


train_score = evaluation_metric(y_train, pred_train)
val_score = evaluation_metric(y_val, pred_val)

print("Train Score : %.4f" % train_score)
print("Validation Score : %.4f" % val_score)

Prediction
Train Score : 0.6801
Validation Score : 0.6664


### 6. Hyper-parameter Tuning

In [64]:
def optimizer(trial, X, y, K):
    # 조절할 hyper-parameter 조합을 적어줍니다.
    max_depth = trial.suggest_int('max_depth', 5, 15)
    colsample_bynode = trial.suggest_float('colsample_bynode', 0.5, 0.8)
    reg_lambda = trial.suggest_float('reg_lambda', 0.5, 5.0)
    n_estimators = trial.suggest_int('n_estimators', 50, 2000)
    learning_rate = trial.suggest_float('learning_rate', 0.01, 0.3)

    # 원하는 모델을 지정합니다, optuna는 시간이 오래걸리기 때문에 저는 보통 RF로 일단 테스트를 해본 뒤에 LGBM을 사용합니다.
    model = XGBClassifier(
        max_depth=max_depth,
        colsample_bynode=colsample_bynode,
        reg_lambda=reg_lambda,
        n_estimators=n_estimators,
        learning_rate=learning_rate,
        random_state=61,
        tree_method='hist',
        enable_categorical=True,
    )

    # K-Fold Cross validation을 구현합니다.
    folds = StratifiedKFold(n_splits=K, random_state=61, shuffle=True)
    losses = []

    for train_idx, val_idx in folds.split(X, y):
        X_train = X.iloc[train_idx, :]
        y_train = y.iloc[train_idx]

        X_val = X.iloc[val_idx, :]
        y_val = y.iloc[val_idx]

        model.fit(X_train, y_train)
        preds = model.predict(X_val)
        loss = evaluation_metric(y_val, preds)
        losses.append(loss)


    # K-Fold의 평균 loss값을 돌려줍니다.
    return np.mean(losses)

In [65]:
K = 5   # Kfold 수
opt_func = partial(optimizer, X=X, y=y, K=K)

study = optuna.create_study(direction="maximize") # 최소/최대 어느 방향의 최적값을 구할 건지.
study.optimize(opt_func, n_trials=50)

[I 2023-10-19 02:27:05,748] A new study created in memory with name: no-name-15ccb2eb-adf4-4c4d-8867-5a66978adfed
[I 2023-10-19 02:27:34,106] Trial 0 finished with value: 0.6619712621672823 and parameters: {'max_depth': 6, 'colsample_bynode': 0.7377810247213195, 'reg_lambda': 0.7052452702564933, 'n_estimators': 1347, 'learning_rate': 0.022253594404113296}. Best is trial 0 with value: 0.6619712621672823.
[I 2023-10-19 02:27:56,685] Trial 1 finished with value: 0.6555666929575741 and parameters: {'max_depth': 11, 'colsample_bynode': 0.6605602936965537, 'reg_lambda': 4.849863580030156, 'n_estimators': 342, 'learning_rate': 0.2866384107016313}. Best is trial 0 with value: 0.6619712621672823.
[I 2023-10-19 02:30:20,938] Trial 2 finished with value: 0.6491230220045756 and parameters: {'max_depth': 13, 'colsample_bynode': 0.7264856041848916, 'reg_lambda': 4.343899109477942, 'n_estimators': 1909, 'learning_rate': 0.19206908603114892}. Best is trial 0 with value: 0.6619712621672823.
[I 2023-10-

In [66]:
# optuna가 시도했던 모든 실험 관련 데이터
study.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_colsample_bynode,params_learning_rate,params_max_depth,params_n_estimators,params_reg_lambda,state
0,0,0.661971,2023-10-19 02:27:05.752472,2023-10-19 02:27:34.106394,0 days 00:00:28.353922,0.737781,0.022254,6,1347,0.705245,COMPLETE
1,1,0.655567,2023-10-19 02:27:34.108527,2023-10-19 02:27:56.684503,0 days 00:00:22.575976,0.660560,0.286638,11,342,4.849864,COMPLETE
2,2,0.649123,2023-10-19 02:27:56.686503,2023-10-19 02:30:20.937871,0 days 00:02:24.251368,0.726486,0.192069,13,1909,4.343899,COMPLETE
3,3,0.650949,2023-10-19 02:30:20.940719,2023-10-19 02:31:30.174115,0 days 00:01:09.233396,0.568987,0.284471,7,1821,4.901384,COMPLETE
4,4,0.649153,2023-10-19 02:31:30.175698,2023-10-19 02:32:57.376025,0 days 00:01:27.200327,0.639058,0.250407,12,1313,1.334971,COMPLETE
5,5,0.653892,2023-10-19 02:32:57.378173,2023-10-19 02:33:55.986407,0 days 00:00:58.608234,0.713914,0.139013,7,1544,0.987812,COMPLETE
6,6,0.653542,2023-10-19 02:33:55.989473,2023-10-19 02:34:39.434516,0 days 00:00:43.445043,0.668630,0.211455,6,1388,3.569923,COMPLETE
7,7,0.652430,2023-10-19 02:34:39.436957,2023-10-19 02:35:21.631533,0 days 00:00:42.194576,0.762524,0.205288,6,1425,2.160020,COMPLETE
8,8,0.652852,2023-10-19 02:35:21.633534,2023-10-19 02:36:06.475640,0 days 00:00:44.842106,0.629149,0.239154,12,577,4.684517,COMPLETE
9,9,0.649374,2023-10-19 02:36:06.477809,2023-10-19 02:37:49.296168,0 days 00:01:42.818359,0.638776,0.128848,13,1532,0.765550,COMPLETE


In [67]:
print("Best Score: %.4f" % study.best_value) # best score 출력
print("Best params: ", study.best_trial.params) # best score일 때의 하이퍼파라미터들

Best Score: 0.6665
Best params:  {'max_depth': 5, 'colsample_bynode': 0.6961189200064242, 'reg_lambda': 2.5677242247002603, 'n_estimators': 51, 'learning_rate': 0.09786684812197355}


In [68]:
# 실험 기록 시각화
optuna.visualization.plot_optimization_history(study)

In [69]:
# hyper-parameter들의 중요도
optuna.visualization.plot_param_importances(study)

### 7. 테스트 및 제출 파일 생성

In [70]:
# Make KFold OOF prediction
def oof_preds(best_model):

    # make KFold
    folds = StratifiedKFold(n_splits=K, random_state=61, shuffle=True)
    final_preds = []
    losses = []
    oof = np.full(len(X), np.nan)
    # fitting with best_model
    for i, (train_idx, val_idx) in enumerate(folds.split(X, y)):
        X_train = X.iloc[train_idx, :]
        y_train = y.iloc[train_idx]
        X_val = X.iloc[val_idx, :]
        y_val = y.iloc[val_idx]

        print(f"========== Fold {i+1} ==========")
        best_model.fit(X_train, y_train)
        preds = best_model.predict_proba(X_val)[:, 1]
        oof[val_idx] = preds
        test_preds = best_model.predict_proba(test)[:, 1]
        final_preds.append(test_preds)
        loss = evaluation_metric(y_val, preds)

        losses.append(loss)

    avg_loss = np.mean(losses)
    print(f"Loss : {avg_loss:.4f}")
    return final_preds, oof

In [71]:
test.info() # 결측치 없음.

<class 'pandas.core.frame.DataFrame'>
Index: 67842 entries, 101763 to 169604
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   loc                67842 non-null  float64 
 1   v(g)               67842 non-null  float64 
 2   ev(g)              67842 non-null  float64 
 3   iv(g)              67842 non-null  float64 
 4   v                  67842 non-null  float64 
 5   d                  67842 non-null  float64 
 6   i                  67842 non-null  float64 
 7   e                  67842 non-null  float64 
 8   b                  67842 non-null  float64 
 9   t                  67842 non-null  float64 
 10  lOCode             67842 non-null  int64   
 11  lOComment          67842 non-null  int64   
 12  locCodeAndComment  67842 non-null  int64   
 13  uniq_Op            67842 non-null  float64 
 14  uniq_Opnd          67842 non-null  float64 
 15  lOBlank_n          67842 non-null  category
 16  n_n

In [72]:
## X_test 만들기 : 앞서했던 전처리를 동일하게 적용해주면 됨.


In [73]:
best_params = study.best_trial.params

# define best model
best_model = XGBClassifier(**best_params, random_state=61, tree_method='hist', enable_categorical=True,)

# model finalization : 가장 일반적으로 좋은 예측 성능을 냈던 모델로, 전체 데이터 트레이닝.

preds, oof = oof_preds(best_model)
preds = np.mean(preds, axis=0)
preds

========== Fold 1 ==========
========== Fold 2 ==========
========== Fold 3 ==========
========== Fold 4 ==========
========== Fold 5 ==========
Loss : 0.7911


array([0.26471943, 0.21795687, 0.66364557, ..., 0.15508252, 0.10903223,
       0.7923894 ], dtype=float32)

In [74]:
submission['defects'] = preds

In [75]:
submission.to_csv(base_path+'preds/xgboost.csv')

In [76]:
oof_df = pd.DataFrame({
    'xgboost_oof': oof
})
oof_df.head()

,xgboost_oof
0,0.090458
1,0.055308
2,0.051901
3,0.095929
4,0.122568


In [77]:
oof_df.to_csv(base_path+'oof/xgboost.csv', index_label='id', header=['xgboost_oof'])

In [78]:
pd.DataFrame({'importance':best_model.feature_importances_,'column':test.columns}).sort_values(by='importance', ascending=False)

,importance,column
21,0.665631,outlier_count
17,0.154951,branchCount_n
0,0.057944,loc
8,0.035403,b
19,0.026022,total_Op_n
15,0.008913,lOBlank_n
14,0.008393,uniq_Opnd
1,0.007044,v(g)
4,0.006454,v
3,0.004163,iv(g)
